# Data Extraction

Our client has decided to give us some prior transaction data to work with on this project. The data can be found be going to the following URL: https://archive.ics.uci.edu/ml/datasets/Online+Retail+II.

The dataset is provided in xlsx format and is ~45MB in size. To limit the network bandwidth required on the initial clone of this repository, the `project` helper command was created. After cloning this repository, one can extract the data, and compile it into a csv file (for easier/faster loading) by running the command `project init` from the project root in a terminal.

# Data Cleaning

Now that we've got our data extracted we'll load it up and start the cleaning process. This is very important as it will heavily effect our modeling if we have major inconsistencies in our dataset. I don't expect their to be many errors, or missing values since this is transaction data (hopefully with no input by humans).

Below is the **attribute information** provided to us by our client, for reference.

- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code 
  starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# load data
df = pd.read_csv("../data/raw/data.csv")

In [3]:
# print first 5 rows
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/01/09 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/01/09 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/01/09 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/01/09 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/01/09 07:45,1.25,13085.0,United Kingdom


In [4]:
# print dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Looking at the info output for our dataframe, there are two data columns which have missing values. The `Description` and `Customer ID` columns.

The `Description` column has a rather low amount of missing values about 5k, we could possibly **impute those values using the stock code to match the items with their correct description**.

The `Customer ID` column has ~250k missing values, which is quite excessive. We'll definitely want to try and **impute these values, using the invoice to see which customers are attached to other products on that same invoice**.

## Missing Values

We'll start off with our `Description` column, and try to impute the missing values using the StockCode to index the products description elsewhere.

### Description

In [5]:
# make dataframe of unique stock with description
stock_df = df.dropna(subset=["Description"])  # drop rows with missing description
stock_df = stock_df.loc[~stock_df.StockCode.duplicated(), ["StockCode", "Description"]]  # select unique stock
stock_df = stock_df.set_index("StockCode")
stock_df.head()

,Description
StockCode,
85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
79323P,PINK CHERRY LIGHTS
79323W,WHITE CHERRY LIGHTS
22041,"RECORD FRAME 7"" SINGLE SIZE"
21232,STRAWBERRY CERAMIC TRINKET BOX


In [6]:
# join the data
dfxstock_df = df.join(stock_df, on="StockCode", lsuffix="_original")
dfxstock_df.head()

,Invoice,StockCode,Description_original,Quantity,InvoiceDate,Price,Customer ID,Country,Description
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/01/09 07:45,6.95,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,PINK CHERRY LIGHTS,12,12/01/09 07:45,6.75,13085.0,United Kingdom,PINK CHERRY LIGHTS
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/01/09 07:45,6.75,13085.0,United Kingdom,WHITE CHERRY LIGHTS
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/01/09 07:45,2.10,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/01/09 07:45,1.25,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX


In [7]:
# are all of the missing values filled now?
dfxstock_df.loc[dfxstock_df.Description.isna(), "StockCode"].value_counts()

gift_0001_90    2
84494B          2
21502           2
37477D          2
84933D          2
               ..
16240           1
16210           1
35980A          1
21029           1
17013A          1
Name: StockCode, Length: 355, dtype: int64

Our dataset still contains stock which has no description, about 355 unique stockcodes have no description. Since our objective for this project doesn't rely on the description we can essentially drop this column when modeling. It would prove useful however if we were doing stock analysis and wanted to predict which items to have in stock throughout a given year. Since we can't impute all of the values, we'll stay with the ones we have imputed and then **after EDA we'll drop the column since we don't need it for modeling**.

In [8]:
# drop the old description
df = dfxstock_df.drop(columns=["Description_original"], errors="ignore")
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
0,489434,85048,12,12/01/09 07:45,6.95,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,12,12/01/09 07:45,6.75,13085.0,United Kingdom,PINK CHERRY LIGHTS
2,489434,79323W,12,12/01/09 07:45,6.75,13085.0,United Kingdom,WHITE CHERRY LIGHTS
3,489434,22041,48,12/01/09 07:45,2.10,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,24,12/01/09 07:45,1.25,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX


### Customer ID

Our dataset has about 250k missing Customer ID's, we'll do a similar process as when trying to impute product descriptions.

In [9]:
# unique invoice - Customer ID dataframe
invoice_df = df.dropna(subset=["Customer ID"])
invoice_df = invoice_df.drop_duplicates(subset=["Invoice", "Customer ID"])[["Invoice", "Customer ID"]]
invoice_df = invoice_df.set_index("Invoice")
invoice_df.head()

,Customer ID
Invoice,
489434,13085.0
489435,13085.0
489436,13078.0
489437,15362.0
489438,18102.0


In [10]:
# Are there any invoices with 2 Customer's
invoice_df.index.duplicated().sum()

0

In [11]:
# join the data together
dfxinvoice_df = df.join(invoice_df, on="Invoice", lsuffix="_original")
dfxinvoice_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID_original,Country,Description,Customer ID
0,489434,85048,12,12/01/09 07:45,6.95,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS,13085.0
1,489434,79323P,12,12/01/09 07:45,6.75,13085.0,United Kingdom,PINK CHERRY LIGHTS,13085.0
2,489434,79323W,12,12/01/09 07:45,6.75,13085.0,United Kingdom,WHITE CHERRY LIGHTS,13085.0
3,489434,22041,48,12/01/09 07:45,2.10,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE",13085.0
4,489434,21232,24,12/01/09 07:45,1.25,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX,13085.0


In [12]:
# do we still have missing customer ID's
dfxinvoice_df["Customer ID"].isna().sum()

243007

We still have the same number of missing Customer ID's we had before we tried to impute them. This means that a number of invoices have no Customer ID attached to them. Looking at our data dictionary, some invoices have a prefix of 'c' to indicate they are a cancellation. Maybe this is what is hindering our imputation.

In [13]:
# how many cancellation invoices with missing Customer ID
df.loc[df["Customer ID"].isna(), "Invoice"].str.match('c').sum()

0

In [14]:
# what are the invoices with missing Customer ID's
df.loc[df["Customer ID"].isna(), "Invoice"].value_counts()

537434    1350
538071    1304
537638    1202
537237    1194
536876    1186
          ... 
496785       1
525339       1
497865       1
497118       1
547526       1
Name: Invoice, Length: 8752, dtype: int64

From the looks of it, our rows with missing Customer ID's, those invoices do not have a Customer ID attached to them at all. For our modeling we'll want to drop these rows, as we will be grouping by Customer ID.

## Erroneous Values

We now know what we'll be doing with our missing values, next we have to inspect the continuous values `Quantity` and `Price` to verify there aren't any erroneous values. Since this is multi-year data I expect that prices may change marginally for the same products, so our main focus will be in finding any extreme anomaly values. 

In [15]:
# see that stats for Quantity and Price
df[["Quantity", "Price"]].describe().applymap(lambda x: f"{x:,.3f}")

,Quantity,Price
count,"1,067,371.000","1,067,371.000"
mean,9.939,4.649
std,172.706,123.553
min,"-80,995.000","-53,594.360"
25%,1.000,1.250
50%,3.000,2.100
75%,10.000,4.150
max,"80,995.000","38,970.000"


Instantly one can see the minimum and maximum values for both Quantity and Price seem like anomalies. We should definitely check any values which appear outside of the IQR range, and verify they are correct. Another thing is that we apparently have negative prices and quantities. This may be due to cancellations/discounts, however, this means that price and quantity can't be negative at the same time, only one or the other.

In [16]:
# Lets find any rows with a negative value in price or quantity
price_mask = df.Price < 0
quant_mask = df.Quantity < 0
neg_df = df[np.logical_xor(price_mask, quant_mask)]  # xor because we want either price or quantity to be neg not both
neg_df

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
178,C489449,22087,-12,12/01/09 10:33,2.95,16321.0,Australia,PAPER BUNTING WHITE LACE
179,C489449,85206A,-6,12/01/09 10:33,1.65,16321.0,Australia,CREAM FELT EASTER EGG BASKET
180,C489449,21895,-4,12/01/09 10:33,4.25,16321.0,Australia,POTTING SHED SOW 'N' GROW SET
181,C489449,21896,-6,12/01/09 10:33,2.10,16321.0,Australia,POTTING SHED TWINE
182,C489449,22083,-12,12/01/09 10:33,2.95,16321.0,Australia,PAPER CHAIN KIT RETRO SPOT
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,-11,12/09/11 09:57,0.83,14397.0,United Kingdom,ZINC T-LIGHT HOLDER STARS SMALL
1067002,C581499,M,-1,12/09/11 10:28,224.69,15498.0,United Kingdom,Manual
1067176,C581568,21258,-5,12/09/11 11:57,10.95,15311.0,United Kingdom,VICTORIAN SEWING BOX LARGE
1067177,C581569,84978,-1,12/09/11 11:58,1.25,17315.0,United Kingdom,HANGING HEART JAR T-LIGHT HOLDER


In [17]:
# invoices with both neg price & quantity
(price_mask & quant_mask).sum()

0

In [18]:
# lets see if all 22955 rows are a part of a cancellation invoice
cancels = neg_df.Invoice.str.match("c", case=False)
neg_df[~cancels]

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
263,489464,21733,-96,12/01/09 10:52,0.0,NaN,United Kingdom,RED HANGING HEART T-LIGHT HOLDER
283,489463,71477,-240,12/01/09 10:52,0.0,NaN,United Kingdom,short
284,489467,85123A,-192,12/01/09 10:53,0.0,NaN,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
470,489521,21646,-50,12/01/09 11:44,0.0,NaN,United Kingdom,NaN
3114,489655,20683,-44,12/01/09 05:26,0.0,NaN,United Kingdom,RAIN GIRL CHILDS UMBRELLA
...,...,...,...,...,...,...,...,...
1060794,581210,23395,-26,12/07/11 06:36,0.0,NaN,United Kingdom,BELLE JARDINIERE CUSHION COVER
1060796,581212,22578,-1050,12/07/11 06:38,0.0,NaN,United Kingdom,WOODEN STAR CHRISTMAS SCANDINAVIAN
1060797,581213,22576,-30,12/07/11 06:38,0.0,NaN,United Kingdom,SWALLOW WOODEN CHRISTMAS DECORATION
1062371,581226,23090,-338,12/08/11 09:56,0.0,NaN,United Kingdom,VINTAGE GLASS T-LIGHT HOLDER


Looks like a subset of the rows with a negative value in either quantity or price are not cancellations. The price appears to be 0 for some of these, which is interesting. I would've expected them to be discounts if our quantity was positive, and our price was negative. These could also be purchases made by a customer before inventory was available. Regardless let's get a closer look.

In [19]:
# descriptive stats of neg_df non-cancels
neg_df[~cancels].describe()

,Quantity,Price,Customer ID
count,3462.000000,3462.000000,0.0
mean,-165.534373,-45.833663,NaN
std,672.651944,1377.140628,NaN
min,-9600.000000,-53594.360000,NaN
25%,-81.750000,0.000000,NaN
50%,-26.000000,0.000000,NaN
75%,-8.000000,0.000000,NaN
max,1.000000,0.000000,NaN


None of these transactions have a customer ID attached to them either, our price is always negative, and the quantity never exceeds 1. Let's spot those transactions with a quantity of 1.

We'll move forward assuming the transactions with a negative quantity and 0 price are inventory corrections, most likely due to a return. We'll drop these since we aren't doing any inventory analysis at this time, and these transactions only effect inventory.

In [20]:
# transactions with a quantity of 1
neg_df[neg_df.Quantity ==1]

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
179403,A506401,B,1,04/29/10 01:36,-53594.36,NaN,United Kingdom,Adjust bad debt
276274,A516228,B,1,07/19/10 11:24,-44031.79,NaN,United Kingdom,Adjust bad debt
403472,A528059,B,1,10/20/10 12:04,-38925.87,NaN,United Kingdom,Adjust bad debt
825444,A563186,B,1,08/12/11 02:51,-11062.06,NaN,United Kingdom,Adjust bad debt
825445,A563187,B,1,08/12/11 02:52,-11062.06,NaN,United Kingdom,Adjust bad debt


[Bad Debts](https://help-sage50.na.sage.com/en-ca/core/2019/Content/Customers_Sales/CustomerPayments/AboutBadDebts.htm): The amount not paid when a customer fails to pay all or part of what is owed. A bad debt is a cost of selling on credit and is considered an expense.

It looks like there was a duplicate entry for a bad debt transaction, 825444 & 825445, as they both are the same except a minute apart, We'll keep it in though, since there is no other indication it could be an error other than it appears to be the same with a minute offset.

Previously we said we'd drop the rows with a missing Customer ID, but it looks like we'll have to fix that, as there is obviously some values that reduce the total revenue grossed. To fix this we'll group by invoice, and find the total for each invoice, then we'll join our original dataset to get our Invoice Date, Country and Customer ID.

In [21]:
# groupby invoice and get total sum
df["total"] = df.Quantity * df.Price
df["invoice_"] = df.Invoice.map(lambda s: s[1:] if s[0] in "AC" else s).astype("int")
i_totals = df.groupby("invoice_").sum()["total"]
i_totals

invoice_
489434     505.30
489435     145.80
489436     630.33
489437     310.75
489438    2286.24
           ...   
581583     124.60
581584     140.64
581585     329.05
581586     339.20
581587     267.45
Name: total, Length: 53628, dtype: float64

In [22]:
# merge the two datasets
data = pd.merge(i_totals, df.sort_values("InvoiceDate"), how="left", left_on="invoice_", right_on="invoice_")
data = data.drop_duplicates(subset=["invoice_"])
data = data[["invoice_", "total_x", "Invoice", "InvoiceDate", "Customer ID", "Country"]]
data.columns = ["invoice_num", "total", "invoice_name", "invoice_date", "customer_id", "country"]
data

,invoice_num,total,invoice_name,invoice_date,customer_id,country
0,489434,505.30,489434,12/01/09 07:45,13085.0,United Kingdom
8,489435,145.80,489435,12/01/09 07:46,13085.0,United Kingdom
12,489436,630.33,489436,12/01/09 09:06,13078.0,United Kingdom
31,489437,310.75,489437,12/01/09 09:08,15362.0,United Kingdom
54,489438,2286.24,489438,12/01/09 09:24,18102.0,United Kingdom
...,...,...,...,...,...,...
1067326,581583,124.60,581583,12/09/11 12:23,13777.0,United Kingdom
1067328,581584,140.64,581584,12/09/11 12:25,13777.0,United Kingdom
1067330,581585,329.05,581585,12/09/11 12:31,15804.0,United Kingdom
1067351,581586,339.20,581586,12/09/11 12:49,13113.0,United Kingdom


In [23]:
# save our data
data.to_csv("../data/interim/data.csv")

Looks like our interim data is all cleaned and ready to continue in the pipeline, we'll next do some EDA, and any additional reshaping before we do our modeling.